In [1]:
from itertools import chain # For Efficient Looping
import nltk # Our angel ;P

import sklearn #Import scikit learnt for stats functions especiallye val metrics
import scipy.stats
#For later hyperparameter tuning

from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

import sklearn_crfsuite #Our savior ;)
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

import eli5 #pretty metrics 

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [3]:
#Dependencies for POS Tagging
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\SHEEBA
[nltk_data]     CHANDER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\SHEEBA CHANDER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
#Sample POS Tags
tokens=nltk.word_tokenize("Texas economy expanded due to increase in oil prices.")
print(nltk.pos_tag(tokens))

[('Texas', 'NNP'), ('economy', 'NN'), ('expanded', 'VBD'), ('due', 'JJ'), ('to', 'TO'), ('increase', 'VB'), ('in', 'IN'), ('oil', 'NN'), ('prices', 'NNS'), ('.', '.')]


In [5]:
#Read sentences from file, POS Tag and write back to file
file_object  = open("FNER_data_train.txt", "r")
data_train = file_object.read()
tokens=nltk.word_tokenize(data_train)
file = open("tagged_file.txt", "w")
file.write(str(nltk.pos_tag(tokens)))

20079

In [ ]:
#Open the file tagged with NER tags to train on it.
file_train = open("tagged_train", "r")
sent_train=list(file_train.read())

In [8]:
#Functions to convert to features and labels. 
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
jar = './fner/stanford-ner.jar'
model = './fner/classifiers/english.all.3class.distsim.crf.ser.gz'

# Prepare NER tagger with english model
ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')
sentence = 'Austin added more than 27,300 net new jobs in 2017.'
# Tokenize: Split sentence into words
words = nltk.word_tokenize(sentence)

# Run NER tagger on words
ner_tagger.tag(words)

In [ ]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
path = 'F:'
st = StanfordNERTagger(path+'/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz')

text = 'Austin added more than 27,300 net new jobs in 2017.'

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)

print(classified_text)

In [35]:
#Splitting into Train and Test for CRF based NER

X_train = [sent2features(s) for s in sent_train]
y_train = [sent2labels(s) for s in sent_train]

X_test = [sent2features(s) for s in sent_test]
y_test = [sent2labels(s) for s in sent_test]

In [36]:
####--------############
Encoder = LabelEncoder()
trainDF = pd.DataFrame()
labelsDF = pd.DataFrame()
testDF = pd.DataFrame()
valuesDF = pd.DataFrame()
col1 = []
col2 = []
col3 = []
for i in range(0,len(sent_train)):
    for j in range(0,len(sent_train[i])):
        col1.append(sent_train[i][j][0])
        col2.append(sent_train[i][j][1])
        col3.append(sent_train[i][j][2])
X = Encoder.fit_transform(col1)
Y = Encoder.fit_transform(col2)
Z = Encoder.fit_transform(col3)
trainDF = trainDF.append([X,Y])
trainDF = trainDF.transpose()
labelsDF = labelsDF.append([Z])
labelsDF = labelsDF.transpose()

In [37]:
testDF = pd.DataFrame()
labels_testDF = pd.DataFrame()
col1 = []
col2 = []
col3 = []
for i in range(0,len(sent_test)):
    for j in range(0,len(sent_test[i])):
        col1.append(sent_test[i][j][0])
        col2.append(sent_test[i][j][1])
        col3.append(sent_test[i][j][2])
X = Encoder.fit_transform(col1)
Y = Encoder.fit_transform(col2)
Z = Encoder.fit_transform(col3)
testDF = testDF.append([X,Y])
testDF = testDF.transpose()
labels_testDF = labels_testDF.append([Z])
labels_testDF = labels_testDF.transpose()

In [49]:
labs = ['B-LOC', 'SENTIMENT', 'NUM', 'B-WFIN', 'B-WECO', 'INTENSITY', 'SFIN', 'B-ORG', 'I-WECO','O']
labels = Encoder.fit_transform(labs)
labels

array([0, 8, 6, 3, 2, 5, 9, 1, 4, 7], dtype=int64)

In [52]:
#########--------############
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(trainDF,labelsDF,labels)
nb.get_params()
nb.score(testDF, labels_testDF)

0.03389830508474576

In [42]:
#Running the CRF algorithm on our sentences
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [43]:
#Remove the "Other" Class
labels = list(crf.classes_)
labels.remove('O')
labels

['B-LOC',
 'SENTIMENT',
 'NUM',
 'B-WFIN',
 'B-WECO',
 'I-WFIN',
 'INTENSITY',
 'SFIN',
 'B-ORG',
 'I-ORG',
 'I-WECO']

In [44]:
#Predict tags on test data and evaluate accuracy
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

C:\Users\SHEEBA CHANDER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\SHEEBA CHANDER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.8421717171717171

In [19]:
#results - Evaluation metrics on individual tags
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

C:\Users\SHEEBA CHANDER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\SHEEBA CHANDER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

       B-LOC      1.000     0.500     0.667         2
       B-ORG      0.000     0.000     0.000         0
       I-ORG      0.000     0.000     0.000         0
      B-WECO      0.500     0.250     0.333         4
      I-WECO      1.000     1.000     1.000         1
      B-WFIN      0.000     0.000     0.000         0
      I-WFIN      0.000     0.000     0.000         0
   SENTIMENT      0.800     1.000     0.889         4
        SFIN      1.000     1.000     1.000         1
   INTENSITY      0.000     0.000     0.000         0
         NUM      1.000     1.000     1.000         6

   micro avg      0.875     0.778     0.824        18
   macro avg      0.482     0.432     0.444        18
weighted avg      0.844     0.778     0.790        18



In [45]:
#Check out most likely transitions
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Order of Most Likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nOrder of Most unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Order of Most Likely transitions:
B-WECO -> I-WECO  2.335394
B-WFIN -> I-WFIN  2.122663
I-WECO -> I-WECO  1.728865
I-WFIN -> I-WFIN  1.686705
B-ORG  -> I-ORG   1.602387
I-ORG  -> I-ORG   1.536221
O      -> O       1.269062
INTENSITY -> SENTIMENT 1.200200
SENTIMENT -> O       0.964744
O      -> B-WECO  0.908758
NUM    -> SFIN    0.861125
SENTIMENT -> I-WFIN  0.774482
O      -> B-WFIN  0.723029
B-LOC  -> O       0.651999
SENTIMENT -> NUM     0.619181
SFIN   -> NUM     0.589059
O      -> B-LOC   0.572464
NUM    -> O       0.565094
O      -> INTENSITY 0.546092
B-WECO -> SENTIMENT 0.522133

Order of Most unlikely transitions:
SENTIMENT -> I-WECO  -0.114303
I-ORG  -> O       -0.115629
I-WECO -> I-WFIN  -0.128508
NUM    -> SENTIMENT -0.136950
B-WFIN -> NUM     -0.149022
B-ORG  -> O       -0.164621
B-WFIN -> I-WECO  -0.179597
B-WFIN -> B-WECO  -0.187874
SENTIMENT -> B-WFIN  -0.250079
NUM    -> I-WFIN  -0.276223
I-WFIN -> O       -0.357447
O      -> SENTIMENT -0.376658
SENTIMENT -> INTENSITY -0

In [46]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
3.518580 NUM      postag:CD
3.518580 NUM      postag[:2]:CD
2.562263 O        bias
1.877506 B-WFIN   -1:word.lower():anticipated
1.793603 B-WFIN   word[-2:]:es
1.755335 INTENSITY postag[:2]:RB
1.620393 SENTIMENT word.lower():expansion
1.594752 SFIN     word.lower():%
1.594752 SFIN     word[-3:]:%
1.594752 SFIN     word[-2:]:%
1.508285 O        postag:DT
1.508285 O        postag[:2]:DT
1.504625 B-LOC    word.lower():austin
1.504625 B-LOC    word[-3:]:tin
1.467190 B-LOC    word[-2:]:in
1.452299 O        postag:IN
1.452299 O        postag[:2]:IN
1.350638 SENTIMENT postag[:2]:VB
1.342015 SENTIMENT word.lower():up
1.342015 SENTIMENT word[-3:]:up
1.342015 SENTIMENT word[-2:]:up
1.325924 B-WFIN   -1:word.istitle()
1.306135 SENTIMENT word[-2:]:ed
1.269889 O        -1:word.lower():a
1.218740 SENTIMENT word.lower():increased
1.196783 SENTIMENT word.lower():growth
1.196783 SENTIMENT word[-3:]:wth
1.191381 O        -1:postag[:2]:NN
1.185784 B-WFIN   +1:word.lower():collections
1.1624

In [47]:
######-----#######
eli5.show_weights(crf, top=10)

From \ To,INTENSITY,NUM,O,SENTIMENT,SFIN,B-LOC,B-ORG,I-ORG,B-WECO,I-WECO,B-WFIN,I-WFIN
INTENSITY,0.0,0.0,0.0,1.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.052
NUM,0.0,0.0,0.565,-0.137,0.861,0.0,0.0,0.0,0.083,-0.059,0.0,-0.276
O,0.546,0.0,1.269,-0.377,0.0,0.572,0.019,-0.801,0.909,-1.045,0.723,-1.599
SENTIMENT,-0.388,0.619,0.965,0.0,0.0,0.0,0.0,0.0,0.251,-0.114,-0.25,0.774
SFIN,0.0,0.589,0.0,0.111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.106
B-LOC,0.0,0.0,0.652,0.084,0.0,0.0,0.0,-0.688,0.0,0.0,-0.001,0.0
B-ORG,0.0,0.0,-0.165,0.0,0.0,0.0,0.0,1.602,0.0,0.0,0.0,0.0
I-ORG,0.0,0.0,-0.116,-0.075,0.0,0.0,0.0,1.536,0.0,0.0,0.0,0.0
B-WECO,0.0,-0.103,-0.1,0.522,0.0,0.0,0.0,0.0,-0.518,2.335,0.126,-0.484
I-WECO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.729,0.0,-0.129


In [34]:
sent_train=[[('Austin', 'NNP','B-LOC'), ('added', 'VBD','SENTIMENT'), ('more', 'JJR','SENTIMENT'), ('than', 'IN','O'), ('27,300', 'CD','NUM'), ('net', 'JJ','B-WFIN'), ('new', 'JJ','O'), ('jobs', 'NNS','B-WECO'), ('in', 'IN','O'), ('2017', 'CD','NUM'), ('.', '.','O')],
[('Sales', 'NNS','B-WFIN'), ('Taxes', 'NNP','B-WFIN'), ('–', 'VBD','O'), ('Consumer', 'NNP','B-WECO'), ('spending', 'NN','B-WFIN'), ('growth', 'NN','SENTIMENT'), ('(', '(','O'), ('as', 'IN','O'), ('measured', 'VBN','O'), ('by', 'IN','O'), ('sales', 'NNS','B-WFIN'), ('tax', 'NN','B-WFIN'), ('collections', 'NNS','O'), (')', ')','O'), ('reflects', 'VBZ','O'), ('Austin', 'NNP','B-LOC'), ('’', 'NNP','O'), ('s', 'POS','O'), ('population', 'NN','B-WECO'), ('and', 'CC','O'), ('economic', 'JJ','B-WECO'), ('growth', 'NN','SENTIMENT'), ('.', '.','O')],
[('Since', 'IN','O'), ('2009', 'CD','NUM'), (',', ',','O'), ('sales', 'NNS','B-WFIN'), ('tax', 'NN','I-WFIN'), ('revenue', 'NN','I-WFIN'), ('steadily', 'RB','INTENSITY'), ('increased', 'VBD','SENTIMENT'), ('.', '.','O')],
[('Fiscal', 'JJ','B-WFIN'), ('year', 'NN','I-WFIN'), ('2017', 'CD','NUM'), ('experienced', 'VBD','O'), ('a', 'DT','O'), ('2.9', 'CD','NUM'), ('%', 'NN','SFIN'), ('increase', 'NN','SENTIMENT'), ('over', 'IN','O'), ('fiscal', 'JJ','B-WFIN'), ('year', 'NN','I-WFIN'), ('2016', 'CD','NUM'), (',', ',','O'), ('which', 'WDT','O'), ('was', 'VBD','O'), ('a', 'DT','O'), ('4.2', 'CD','NUM'), ('%', 'NN','SFIN'), ('increase', 'NN','SENTIMENT'), ('over', 'IN','O'), ('2015', 'CD','NUM'), ('.', '.','O')],
[('Lower-than', 'JJ','SENTIMENT'), ('anticipated', 'JJ','O'), ('growth', 'NN','B-WFIN'), ('in', 'IN','O'), ('fiscal', 'JJ','B-WFIN'), ('year', 'NN','I-WFIN'), ('2017', 'CD','NUM'), (',', ',','O'), ('despite', 'IN','O'), ('a', 'DT','O'), ('continued', 'JJ','O'), ('population', 'NN','B-WECO'), ('influx', 'NN','SENTIMENT'), (',', ',','O'), ('evidences', 'VBZ','O'), ('the', 'DT','O'), ('effects', 'NNS','O'), ('of', 'IN','O'), ('minor', 'JJ','INTENSITY'), ('gains', 'NNS','SENTIMENT'), ('in', 'IN','O'), ('real', 'JJ','O'), ('income', 'NN','B-WFIN'), ('growth', 'NN','SENTIMENT'), (',', ',','O'), ('but', 'CC','O'), ('it', 'PRP','O'), ('also', 'RB','O'), ('reflects', 'VBZ','O'), ('the', 'DT','O'), ('growing', 'VBG','SENTIMENT'), ('competitive', 'JJ','O'), ('challenge', 'NN','O'), ('presented', 'VBN','O'), ('by', 'IN','O'), ('the', 'DT','O'), ('expansion', 'NN','SENTIMENT'), ('of', 'IN','O'), ('retail', 'JJ','B-WFIN'), ('developments', 'NNS','I-WFIN'), ('in', 'IN','O'), ('Austin', 'NNP','B-LOC'), ('’', 'NNP','O'), ('s', 'VBD','O'), ('neighboring', 'JJ','O'), ('communities', 'NNS','O'), ('.', '.','O')],
[('Total', 'JJ','O'), ('personal', 'JJ','B-WFIN'), ('income', 'NN','I-WFIN'), ('in', 'IN','O'), ('the', 'DT','O'), ('metro', 'NN','O'), ('area', 'NN','O'), ('grew', 'VBD','SENTIMENT'), ('4.5', 'CD','NUM'), ('%', 'NN','SFIN'), ('in', 'IN','O'), ('2017', 'CD','NUM'), ('up', 'IN','SENTIMENT'), ('from', 'IN','O'), ('0.9', 'CD','NUM'), ('%', 'NN','SFIN'), ('in', 'IN','O'), ('2016', 'CD','NUM'), ('.', '.','O')],
[('Tourism', 'NNP','B-WECO'), ('-', ':','O'), ('Austin', 'NN','B-LOC'), ('continues', 'VBZ','O'), ('to', 'TO','O'), ('be', 'VB','O'), ('a', 'DT','O'), ('destination', 'NN','O'), ('for', 'IN','O'), ('both', 'DT','O'), ('business', 'NN','B-WFIN'), ('and', 'CC','O'), ('recreational', 'JJ','O'), ('activities', 'NNS','O'), ('.', '.','O')],
[('Austin', 'NNP','B-LOC'), ('is', 'VBZ','O'), ('known', 'VBN','O'), ('around', 'IN','O'), ('the', 'DT','O'), ('world', 'NN','O'), ('as', 'IN','O'), ('the', 'DT','O'), ('“', 'NNP','O'), ('Live', 'NNP','O'), ('Music', 'NNP','O'), ('Capital', 'NNP','O'), ('of', 'IN','O'), ('the', 'DT','O'), ('World', 'NNP','O'), ('”', 'NN','O'), ('with', 'IN','O'), ('over', 'IN','SENTIMENT'), ('250', 'CD','NUM'), ('live', 'JJ','O'), ('music', 'NN','O'), ('venues', 'NNS','O'), ('.', '.','O')],
[('Austin', 'NNP','B-ORG'), ('Bergstrom', 'NNP','I-ORG'), ('International', 'NNP','I-ORG'), ('Airport', 'NNP','I-ORG'), ('(', '(','O'), ('ABIA', 'NNP','B-ORG'), (')', ')','O'), ('set', 'VBD','O'), ('new', 'JJ','O'), ('annual', 'JJ','O'), ('passenger', 'NN','B-WECO'), ('traffic', 'NN','I-WECO'), ('records', 'NNS','I-WECO'), ('in', 'IN','O'), ('each', 'DT','O'), ('of', 'IN','O'), ('the', 'DT','O'), ('last', 'JJ','O'), ('eight', 'CD','NUM'), ('years', 'NNS','O'), ('.', '.','O')],
[('This', 'DT','O'), ('economic', 'JJ','B-WECO'), ('expansion', 'NN','SENTIMENT'), ('helped', 'VBD','SENTIMENT'), ('the', 'DT','O'), ('unemployment', 'NN','B-WECO'), ('rate', 'NN','I-WECO'), ('fall', 'NN','SENTIMENT'), ('to', 'TO','O'), ('a', 'DT','O'), ('record', 'NN','O'), ('low', 'NN','SENTIMENT'), ('of', 'IN','O'), ('3.8', 'CD','NUM'), ('%', 'NN','SFIN'), ('in', 'IN','O'), ('2017', 'CD','NUM'), ('with', 'IN','O'), ('total', 'JJ','O'), ('new', 'JJ','O'), ('non-farm', 'JJ','B-WECO'), ('jobs', 'NNS','I-WECO'), ('exceeding', 'VBG','SENTIMENT'), ('278,000', 'CD','NUM'), ('.', '.','O')],
[('The', 'DT','O'), ('state', 'NN','O'), ('added', 'VBD','SENTIMENT'), ('9', 'CD','NUM'), ('%', 'NN','SFIN'), ('more', 'JJR','SENTIMENT'), ('jobs', 'NNS','B-WECO'), ('in', 'IN','O'), ('energy', 'NN','O'), ('and', 'CC','O'), ('2.8', 'CD','NUM'), ('%', 'NN','SFIN'), ('more', 'JJR','SENTIMENT'), ('in', 'IN','O'), ('manufacturing', 'NN','O'), ('.', '.','O')],
[('Employee', 'NNP','B-WECO'), ('compensation', 'NN','I-WECO'), ('remained', 'VBD','O'), ('stagnant', 'JJ','INTENSITY'), ('as', 'IN','O'), ('private', 'JJ','B-WFIN'), ('hourly', 'JJ','I-WFIN'), ('earnings', 'NNS','I-WFIN'), ('have', 'VBP','O'), ('increased', 'VBN','SENTIMENT'), ('1.1', 'CD','NUM'), ('%', 'NN','SFIN'), ('statewide', 'NN','O'), ('.', '.','O')], 
[('The', 'DT','O'), ('Austin', 'NNP','B-LOC'), ('Metropolitan', 'NNP','O'), ('area', 'NN','O'), ('is', 'VBZ','O'), ('consistently', 'RB','INTENSITY'), ('recognised', 'VBN','O'), ('as', 'IN','O'), ('most', 'RBS','INTENSITY'), ('profit-oriented', 'JJ','SENTIMENT'), ('with', 'IN','O'), ('a', 'DT','O'), ('growth', 'NN','SENTIMENT'), ('rate', 'NN','I-WFIN'), ('of', 'IN','O'), ('7.5', 'CD','NUM'), ('%', 'NN','SFIN'), ('in', 'IN','O'), ('2017', 'CD','NUM'), ('.', '.','O')], 
[('Austin', 'NNP','B-LOC'), ('is', 'VBZ','O'), ('highly', 'RB','INTENSITY'), ('educated', 'VBN','SENTIMENT'), (',', ',','O'), ('with', 'IN','O'), ('7.5', 'CD','NUM'), ('%', 'NN','SFIN'), ('of', 'IN','O'), ('adults', 'NNS','O'), ('having', 'VBG','O'), ('an', 'DT','O'), ('income', 'NN','B-WFIN'), ('of', 'IN','O'), ('atleast', 'JJ','O'), ('$', '$','SFIN'), ('75,000', 'CD','NUM'), ('and', 'CC','O'), ('an', 'DT','O'), ('average', 'NN','O'), ('of', 'IN','O'), ('$', '$','SFIN'), ('57,000', 'CD','NUM'), ('.', '.','O')], 
[('The', 'DT','O'), ('property', 'NN','B-WFIN'), ('tax', 'NN','I-WFIN'), ('revenue', 'NN','I-WFIN'), ('increased', 'VBN','SENTIMENT'), ('by', 'IN','O'), ('$', '$','SFIN'), ('47.5', 'CD','NUM'), ('million', 'CD','NUM'), ('to', 'TO','O'), ('$', '$','SFIN'), ('107.5', 'CD','NUM'), ('million', 'CD','NUM'), ('.', '.','O')],
[('The', 'DT','O'), ('expenses', 'NNS','B-WFIN'), ('decreased', 'VBN','SENTIMENT'), ('by', 'IN','O'), ('3.9', 'CD','NUM'), ('%', 'NN','SFIN'), ('due', 'JJ','O'), ('to', 'TO','O'), ('an', 'DT','O'), ('increase', 'NN','SENTIMENT'), ('in', 'IN','O'), ('operating', 'NN','B-WECO'), ('and', 'CC','O'), ('maintenance', 'NN','B-WFIN'), ('costs', 'NNS','I-WFIN'), ('over', 'IN','O'), ('the', 'DT','O'), ('2017-19', 'JJ','NUM'), ('period', 'NN','O'), ('.', '.','O')],
[('Per', 'IN','O'), ('the', 'DT','O'), ('Downtown', 'NNP','B-ORG'), ('Austin', 'NNP','I-ORG'), ('Alliance', 'NNP','I-ORG'), (',', ',','O'), ('Austin', 'NNP','B-LOC'), ('has', 'VBZ','O'), ('over', 'RP','SENTIMENT'), ('25.6', 'CD','NUM'), ('million', 'CD','NUM'), ('visitors', 'NNS','O'), ('a', 'DT','O'), ('year', 'NN','O'), ('with', 'IN','O'), ('a', 'DT','O'), ('$', '$','SFIN'), ('7', 'CD','NUM'), ('billion', 'CD','NUM'), ('annual', 'JJ','B-WECO'), ('economic', 'JJ','I-WECO'), ('impact', 'NN','I-WECO'), ('from', 'IN','O'), ('visitor', 'NN','B-WFIN'), ('spending', 'NN','I-WFIN'), ('.', '.','O')]]
sent_test=[[('Since', 'IN','O'), ('2009', 'CD','NUM'), (',', ',','O'), ('sales', 'NNS','B-WFIN'), ('tax', 'NN','I-WFIN'), ('revenue', 'NN','I-WFIN'), ('steadily', 'RB','INTENSITY'), ('increased', 'VBD','SENTIMENT'), ('.', '.','O')],
[('In', 'IN','O'), ('2017', 'CD','NUM'), (',', ',','O'), ('passenger', 'NN','B-WECO'), ('traffic', 'NN','I-WECO'), ('reached', 'VBD','O'), ('13.9', 'CD','NUM'), ('million', 'CD','NUM'), (',', ',','O'), ('an', 'DT','O'), ('11.5', 'CD','NUM'), ('%', 'NN','SFIN'), ('increase', 'NN','SENTIMENT'), ('compared', 'VBN','O'), ('to', 'TO','O'), ('2016', 'CD','NUM'), ('.', '.','O')],
[('Texas', 'NNP','B-LOC'), ('economy', 'NN','B-WECO'), ('expanded', 'VBD','SENTIMENT'), ('due', 'JJ','O'), ('to', 'TO','O'), ('increase', 'VB','SENTIMENT'), ('in', 'IN','O'), ('oil', 'NN','B-WECO'), ('prices', 'NNS','B-WECO'), ('.', '.','O')],
[('Austin', 'NNP','B-LOC'), ('is', 'VBZ','O'), ('known', 'VBN','O'), ('around', 'IN','O'), ('the', 'DT','O'), ('world', 'NN','O'), ('as', 'IN','O'), ('the', 'DT','O'), ('“', 'NNP','O'), ('Live', 'NNP','O'), ('Music', 'NNP','O'), ('Capital', 'NNP','O'), ('of', 'IN','O'), ('the', 'DT','O'), ('World', 'NNP','O'), ('”', 'NN','O'), ('with', 'IN','O'), ('over', 'IN','SENTIMENT'), ('250', 'CD','NUM'), ('live', 'JJ','O'), ('music', 'NN','O'), ('venues', 'NNS','O'), ('.', '.','O')]]